In [ ]:
# what are the steps taht we perform here

# [x] grab the data 
# [] run multiple gauss fits over the date with multiple initialization points 
# [] check what is the result for each run and which one gives the least error
# [] take only 70% of the points 
# [] write the result to the csv

# [] final consolidated table -> made per channel per type of gene

In [ ]:
# want to see the images inline
%matplotlib inline

# imports
# general 
import os
import glob
from functools import reduce
import re
import csv as csv
# scientific 
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model, datasets
from scipy.stats import norm, gamma
from scipy.optimize import curve_fit
from scipy import special

import pandas as pd

from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# simple gamma function
def g_x(x, a, c):
    return x**(a - 1)*np.exp(-x)/special.gamma(a) + c

In [ ]:
# function for loading one data-set
def load_data(file_path, skiprows_=1):
    data = np.loadtxt(file_path, delimiter = '\t', skiprows=skiprows_)
    # print(data.shape) 
    # handles some weird cases, e.g. when there is no data in the file
    if (len(data.shape) < 2):
        data = data[None, :]
    if (data.shape[1] == 0):
        I = np.array([0])
    else:
        I = data[:, -1]
    return I

In [ ]:
# some folder magic 
# folder = '/Volumes/1TB/2018-05-15-12-30-27-SEA12-full-stack/' # folder contains z-corrected spots 
folder = '/Volumes/1TB/2018-06-14-12-36-00-N2-full-stack/'

In [ ]:
# some const params for all graphs
num_bins = 100; 
# graph [xmin, xmax]
xmin = 0
xmax = 3

binwidth = (xmax - xmin)/(num_bins - 1)

In [ ]:
bins = np.arange(xmin, xmax + binwidth, binwidth)
print ('bins: ', bins.shape)

In [ ]:
# for testing 
filename = 'C1-N2_311'
filepath =  folder + "csv-2/" + filename + '.csv'
I = load_data(filepath)

fig = plt.figure(figsize=(8,5))
title = 'C1-N2_311'
plt.title(title)
    
plt.xlabel('intensity')
plt.ylabel('# spots')

print("I_min:", min(I), "I_max:", max(I))

I_res = I
fit_alpha, fit_loc, fit_beta = gamma.fit(I_res)
print(fit_alpha, fit_loc, fit_beta)

plt.hist(I, bins=bins, color='pink', normed=True); # 
# plt.text(0.9*xmax, 0.1, "Total: " + str(I.shape[0]), color='black', bbox=dict(facecolor='white', alpha=1))

info_text = "Total: " + str(I.shape[0])

x_limits = [xmin, xmax]
ymax = np.max(np.histogram(I, bins)[0])
y_limits = [0, ymax]

plt.text(x_limits[1] - (x_limits[1] - x_limits[0])*0.1, y_limits[0] + (y_limits[1] - y_limits[0])*0.04, info_text, color='black', bbox=dict(facecolor='white', alpha=1))
   

x = np.linspace(xmin, xmax, 1000)
y = gamma.pdf(x, fit_alpha, fit_loc, fit_beta)
plt.plot(x,y)

print("peak center:", x[np.argmax(y)])

plt.xlim([xmin, xmax])

# plt.legend(loc = 'upper right')

In [ ]:
# possible labels 
stain = ['DPY-23_EX', 'WDR-5.2', 'MDH-1']
stage = 'E' # only embryos
comment = '' # only empty ones

In [ ]:
# important indices
stain_columns = ['C0_stain', 'C1_stain', 'C2_stain', 'C3_stain', 'C4_stain']
type_columns = ['C0_type', 'C1_type', 'C2_type', 'C3_type', 'C4_type']
stain_prefix = np.array([['C1-', 'C2-', 'C3-', 'C4-', 'C5-']])
ext = '.csv'
filename_column = 'new filename'

In [ ]:
# read the db and parse images that we want to process
df = pd.read_csv(folder + "smFISH-database/N2-Table 1.csv", sep=',', na_values=['']);
df.head()

In [ ]:
# this is general
# filter to have only *good* and *embryo* files
good_indices = np.logical_and((df['stage'] == stage).tolist() , (df['comment'].isnull()).tolist())
good_indices.shape[0]


print(np.sum(good_indices == True))

In [ ]:
# choose necessary stains
dataset1 = []
df_good = (df[type_columns].astype(np.object) == stain[0]).loc[good_indices, :]
row, col = np.where(df_good)
n_samples = df.shape[0]
new_prefix = np.repeat(stain_prefix, n_samples, axis=0)[row, col]
new_filename = df[filename_column].loc[good_indices].as_matrix()[row]
dataset1 = ["{}{}".format(a_, b_) for a_, b_ in zip(new_prefix, new_filename)]

In [ ]:
# choose necessary stains
dataset2 = []
df_good = (df[type_columns].astype(np.object) == stain[1]).loc[good_indices, :]
row, col = np.where(df_good)
n_samples = df.shape[0]
new_prefix = np.repeat(stain_prefix, n_samples, axis=0)[row, col]
new_filename = df[filename_column].loc[good_indices].as_matrix()[row]
dataset2 = ["{}{}".format(a_, b_) for a_, b_ in zip(new_prefix, new_filename)]

In [ ]:
# choose necessary stains
dataset3 = []
df_good = (df[type_columns].astype(np.object) == stain[2]).loc[good_indices, :]
row, col = np.where(df_good)
n_samples = df.shape[0]
new_prefix = np.repeat(stain_prefix, n_samples, axis=0)[row, col]
new_filename = df[filename_column].loc[good_indices].as_matrix()[row]
dataset3 = ["{}{}".format(a_, b_) for a_, b_ in zip(new_prefix, new_filename)]

In [ ]:
print(len(dataset1) + len(dataset2) + len(dataset3))

In [ ]:
def create_title(path, name_id=8):
    # get the name of the initial image
    image_name = path.split("/")[name_id] # was 8
    # print(path.split("/"))
    # create the full title 
    title = image_name[:-4]
    return title
# create_title("/Users/kkolyva/Desktop/n2/N2-results/all/C1-N2_9.csv")

In [ ]:
labels = ['DPY-23_EX'] # ['DPY-23_EX', 'WDR-5.2', 'MDH-1']
color = '#BA5536'
if labels[0] == 'MDH-1':
    color = "#693D3D"

In [ ]:
# actual plotting 

dataset_to_use = dataset1
if labels[0] == 'MDH-1':
    dataset_to_use = dataset3
    
dataset = []
p_dataset = []
for j in range(0, len(dataset_to_use)):
    tmp = folder + "csv-2/" + dataset_to_use[j] + ".csv"
    dataset.append(tmp)
    print(tmp)

In [ ]:
# how good is fitter-meter?
def fitter_meter(y, y_hat):
    return [mean_absolute_error(y,y_hat), np.sqrt(mean_squared_error(y,y_hat))]

In [ ]:
# have to perform this step multiple times and choose the best one 
# perform n_fits with different initial parameters
# n_fits = 10

folder_path = folder + "histograms-2/" + labels[0] + "/"

# actual plotting 
for idx in range(0, len(dataset)):    
    if(not os.path.exists(dataset[idx])):
        # print("doesn't exist")
        continue
        
    try:
        # create the canvas
        fig = plt.figure(figsize=(8,5))
        title = create_title(dataset[idx], name_id=5)
        fig.suptitle(title + " / " + labels[0])

        # load the data and scale it accordingly
        I = load_data(dataset[idx], skiprows_=0)

        # some const params for all graphs
        # num_bins = 100; 
        # graph [xmin, xmax]
        # xmin = np.min(I)
        # xmax = np.max(I)
        # binwidth = (xmax - xmin)/(num_bins - 1)
        # bins = np.arange(xmin, xmax + binwidth, binwidth)

        I_res = I
        # calculate the params for gauss fit
        binned_values, real_bins = np.histogram(I, bins)
        use_median = np.median(I_res)
        # inititally there was use_median/2 
        fit_alpha, fit_loc, fit_beta = gamma.fit(I_res, loc=use_median/2, scale=1/np.max(binned_values))
        # normalization factor
        factor = np.sum(binned_values*np.diff(real_bins))

        plt.hist(I, bins=bins, color=color, label=labels, normed=False)

        x = np.linspace(xmin, xmax, 1000)
        y = gamma.pdf(x, fit_alpha, fit_loc, fit_beta)*factor
        plt.plot(x,y, linewidth=5, color='#66A5AD')
        yhat = gamma.pdf(real_bins, fit_alpha, fit_loc, fit_beta)*factor

        # vertical line for center
        plt.axvline(x=real_bins[np.argmax(yhat)], linestyle="--", linewidth=5, color='#66A5AD')

        if (np.any(np.isnan(yhat))):
            continue

        error = fitter_meter(binned_values, yhat[:-1])

        print("error: L1, L2", error)
        print("peak center:", real_bins[np.argmax(yhat)])

        # reasonable adjustments to make the data look nicer
        plt.xlabel('intensity')
        plt.ylabel('# spots')

        info_text = "Total: " + str(I.shape[0]) + "\n" + "Peak: " +  str('%.2f' % real_bins[np.argmax(yhat)]) + "\n" + "L1: " + str('%.2f' % error[0]) + "\n" + "L2: " +  str('%.2f' % error[1]) 

        x_limits = [xmin, xmax]
        ymax = np.max(np.histogram(I, bins)[0])
        y_limits = [0, ymax]

        plt.text(x_limits[1] - (x_limits[1] - x_limits[0])*0.15, y_limits[1]*0.8, info_text, color='black', bbox=dict(facecolor='white', alpha=1))
        plt.xlim(x_limits)

        # save the peak values for further 
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        plt.savefig(folder_path + title + ".pdf") 
        plt.show()
    except(RuntimeError, TypeError, ValueError):
        print("There was an exception but we\'ll fix it for you")
    # break

### ?plt.hist

In [ ]:
# make the plot with the total number of the detections
folder_path = folder + "total-2/" + labels[0] + "/"
print(folder_path)

result_set = {}

# actual plotting 
for idx in range(0, len(dataset)):    
    if(not os.path.exists(dataset[idx])):
        # print("doesn't exist")
        continue
   
    try:
        print(dataset[idx])

        # load the data and scale it accordingly
        I = load_data(dataset[idx])
        I_res = I

        # some const params for all graphs
        num_bins = 100; 
        # graph [xmin, xmax]
        xmin = 0
        xmax = 3
        binwidth = (xmax - xmin)/(num_bins - 1)
        bins = np.arange(xmin, xmax + binwidth, binwidth)

        binned_values, real_bins = np.histogram(I_res, bins)
        use_median = np.median(I_res)
        # inititally there was use_median/2 
        fit_alpha, fit_loc, fit_beta = gamma.fit(I_res, loc=use_median/2, scale=1/np.max(binned_values))
        # normalization factor
        factor = np.sum(binned_values*np.diff(real_bins))
        x = np.linspace(xmin, xmax, 1000)
        y = gamma.pdf(x, fit_alpha, fit_loc, fit_beta)*factor
        yhat = gamma.pdf(real_bins, fit_alpha, fit_loc, fit_beta)*factor

        if (np.any(np.isnan(yhat))):
            continue

        # should be close to 1
        x_center = real_bins[np.argmax(yhat)]

        # error less than 15%
        print ("error:", np.abs(x_center - 1), "; center:", x_center)
        if (np.abs(x_center - 1) < 0.15):    
            title = create_title(dataset[idx], name_id=5)
            result_set[title] = len(I)   
    except(RuntimeError, TypeError, ValueError, StopIteration):
        print("There was an exception but we\'ll fix it for you")
            
print(list(result_set.values()))        
        
# these are subject to change
xmin = 0
xmax = 4000
binwidth = 100
num_bins = (xmax - xmin) / binwidth + 1;
bins = np.arange(xmin, xmax + binwidth, binwidth)
xlimits = [xmin, xmax]

spots_total = list(result_set.values())

fig = plt.figure(figsize=(8,5))
fig.suptitle(labels[0] + ": Total")
plt.hist(spots_total, bins=bins, color=color, label=labels, normed=False)

plt.xlabel("# spots")
plt.ylabel("# embryos")

df_center = pd.DataFrame(list(result_set.items()), columns=['filename', 'total'])
if not os.path.exists(folder_path):
    os.makedirs(folder_path )
df_center.to_csv(folder_path + labels[0] +"-total.csv", index=False, header=True, encoding='utf-8', mode = 'w' )
plt.savefig(folder_path + labels[0] + "-total.pdf") 
plt.show()
        
print ("DOGE!")  